In [1]:
import pandas as pd

### Import Table from Wikipedia into Datafram

In [55]:
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
nhoods = pd.read_html(link)[0]
nhoods = nhoods[nhoods.Borough != 'Not assigned']
nhoods.Neighbourhood.replace("Not assigned",nhoods.Borough, inplace=True)
nhoods = nhoods.groupby('Postcode', as_index=False).agg({'Borough':'first','Neighbourhood':lambda x:", ".join(x)})

nhoods.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [56]:
nhoods.shape

(103, 3)